In [3]:
import os
from flask import Flask, render_template, request, session, request, redirect, Response
from flask import url_for
import cv2
import uuid
import base64
import json
from utils  import *
import time
import subprocess as sp
import traceback
import numpy as np
import pandas

FPS = 30 
SKIP = 0 
WFPS = 25


app = Flask(__name__)

WASABI = Wasabi()

if not WASABI.bucket_exists("tf-fd-demo"):
    WASABI.s3.create_bucket(Bucket="tf-fd-demo")
    WASABI.set_bucket_policy("tf-fd-demo")
    
base_url = "https://s3.wasabisys.com/{}/".format("tf-fd-demo")

@app.route('/process-video', methods=["POST"])
def process_url():
    
    payload = request.get_json(force=True)
    uid = uuid.uuid4()

    with open("./%s.webm" % uid, "wb") as f:
        video = base64.b64decode(payload["video"].split(",")[1])
        f.write(video)
    
    cap = cv2.VideoCapture("./%s.webm" % uid)
    out = cv2.VideoWriter(
        './%s.mp4' % uid, 
        cv2.VideoWriter_fourcc(*'mp4v'), int(WFPS), 
        (int(cap.get(3)), int(cap.get(4))), True)

    counter = 0
    while True:
        ret, frame = cap.read()
        try:
            if not ret:
                break
        except Exception as e:
            print(e)
            print(traceback.format_exc())

        out.write(frame)
        counter += 1
    cap.release()
    out.release()

    encode_segement = [
        "ffmpeg",
        "-re", "-i", "%s.mp4" % uid,
        "-g", "52",
        "-acodec", "copy",
        "-vcodec", "libx264", 
        "-crf", "27",
        "-preset", "ultrafast",
        "-c:a", "copy",
        "-f", "mp4",
        "%s_tmp.mp4" % uid
    ]
    devnull = open(os.devnull, 'w')
    sp.call(encode_segement, stdout=devnull, stderr=devnull)
    fragment_segement = [
        "./mp4fragment","%s_tmp.mp4" % uid, "%s_out.mp4" % uid
    ]
    sp.call(fragment_segement, stdout=devnull, stderr=devnull)

    out = None
    WASABI.upload_file("%s_out.mp4" % uid, "tf-fd-demo", "%s_out.mp4" % uid)
    
    url = base_url + "%s_out.mp4" % uid

    os.system("rm %s.webm" % uid)
    os.system("rm %s.mp4" % uid)
    os.system("rm %s_tmp.mp4" % uid)
    os.system("rm %s_out.mp4" % uid)
    return json.dumps({"success":True, "URL": url}), 200, {"Content-Type":"application/json"}


@app.route('/', defaults={'path': ''})
@app.route('/<path:path>')
def home(path):
    if path == '':
        page = 'index.html'
    else:
        page = path
    return render_template(page)


if __name__ == '__main__':
    app.run(debug=False, host= '0.0.0.0', threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Sep/2020 18:41:54] "GET / HTTP/1.1" 200 -
[2020-09-22 18:41:55,258] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1935, in

127.0.0.1 - - [22/Sep/2020 18:42:07] "POST /process-video HTTP/1.1" 500 -
[2020-09-22 18:42:10,107] ERROR in app: Exception on /process-video [POST]
Traceback (most recent call last):
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_fu

[2020-09-22 18:42:16,718] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/trueface/.local/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-3-e5cc744ac52d

127.0.0.1 - - [22/Sep/2020 18:46:44] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:45] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:47] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:48] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:49] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:51] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:52] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:55] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:57] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:46:59] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:47:01] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:47:03] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:47:05] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:47:07] "

127.0.0.1 - - [22/Sep/2020 18:49:57] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:49:59] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:01] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:03] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:05] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:08] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:10] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:12] "POST /process-video HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 18:50:13] "POST /process-video HTTP/1.1" 200 -
